In [1]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install datasets
!python -m spacy download en_core_web_sm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Cloning https://github.com/boudinfl/pke.git to /private/var/folders/gf/q45ftd6d5z7c8tcytwj5czh80000gp/T/pip-req-build-gff2883v
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /private/var/folders/gf/q45ftd6d5z7c8tcytwj5czh80000gp/T/pip-req-build-gff2883v
  Resolved https://github.com/boudinfl/pke.git to commit 03d81e440d226407573f61d68ad847fbe015f654
  Preparing metadata (setup.py) ... done
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/o

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: No module named spacy


In [2]:
import spacy
from spacy.tokenizer import _get_regex_pattern

nlp = spacy.load("en_core_web_sm")

# Tokenization fix for in-word hyphens (e.g. 'non-linear' would be kept 
# as one token instead of default spacy behavior of 'non', '-', 'linear')
# https://spacy.io/usage/linguistic-features#native-tokenizer-additions

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

In [3]:
from tqdm.notebook import tqdm
from datasets import load_dataset

# load the inspec dataset
dataset = load_dataset('taln-ls2n/inspec')

# pre-process training and test splits
train = []
for sample in tqdm(dataset['train']):
    train.append(nlp(sample["title"]+". "+sample["abstract"]))
test = []
for sample in tqdm(dataset['test']):
    test.append(nlp(sample["title"]+". "+sample["abstract"]))

No config specified, defaulting to: inspec/raw
Reusing dataset inspec (/Users/boudin-f/.cache/huggingface/datasets/taln-ls2n___inspec/raw/1.1.0/0ae146cabe770846946b3279b4c751efe0aca2dd68b3f24427d4624cd22bb20d)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [4]:
from pke import compute_document_frequency, compute_lda_model
from string import punctuation


# computing LDA and DF counts

compute_document_frequency(
    documents=train,
    output_file='data/inspec.df.gz',
    language='en',              # language of the input files
    normalization='stemming',   # use porter stemmer
    stoplist=list(punctuation), # stoplist (punctuation marks)
    n=5                         # compute n-grams up to 5-grams
)

compute_lda_model(
    documents=train,
    output_file="data/inspec.lda.pickle.gz",
    n_topics=500,               # number of topics
    language='en',              # language of the input files
    stoplist=list(punctuation), # stoplist (punctuation marks)
    normalization='stemming'    # use porter stemmer
)

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
from pke import load_document_frequency_file, load_lda_model

df = load_document_frequency_file(input_file='inspec.df.gz')
lda = load_lda_model(input_file="data/inspec.lda.pickle.gz")

In [12]:
from pke.unsupervised import *

outputs = {}
for model in [FirstPhrases, TextRank, SingleRank, TopicRank, PositionRank, MultipartiteRank, TfIdf, TopicalPageRank]:
    outputs[model.__name__] = []
    
    extractor = model()
    for i, doc in enumerate(tqdm(test)):
        extractor.load_document(input=doc, language='en')
        extractor.grammar_selection(grammar="NP: {<ADJ>*<NOUN|PROPN>+}")
        if model.__name__ in ["TfIdf"]:
            extractor.candidate_weighting(df=df)
        elif model.__name__ in ["TopicalPageRank"]:
            extractor.candidate_weighting(lda_model="data/inspec.lda.pickle.gz")
        else:
            extractor.candidate_weighting()
        outputs[model.__name__].append([u for u,v in extractor.get_n_best(n=5, stemming=True)])

        

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


In [13]:
from nltk.stem.snowball import SnowballStemmer as Stemmer
import numpy as np
    
# populates the references list with stemmed keyphrases
references = []
for sample in tqdm(dataset['test']):
    sample_keyphrases = []
    for keyphrase in sample["keyphrases"]:
        # tokenize keyphrase
        tokens = [token.text for token in nlp(keyphrase)]
        # normalize tokens using Porter's stemming
        stems = [Stemmer('porter').stem(tok.lower()) for tok in tokens]
        sample_keyphrases.append(" ".join(stems))
    references.append(sample_keyphrases)

def evaluate(top_N_keyphrases, references):
    P = len(set(top_N_keyphrases) & set(references)) / len(top_N_keyphrases)
    R = len(set(top_N_keyphrases) & set(references)) / len(references)
    F = (2*P*R)/(P+R) if (P+R) > 0 else 0 
    return (P, R, F)

  0%|          | 0/500 [00:00<?, ?it/s]

In [14]:
# loop through the models
for model in outputs:
    
    # compute the P, R, F scores for the model
    scores = []
    for i, output in enumerate(tqdm(outputs[model])):
        scores.append(evaluate(output, references[i]))
    
    # compute the average scores
    avg_scores = np.mean(scores, axis=0)
    
    # print out the performance of the model
    print("Model: {} P@5: {:.3f} R@5: {:.3f} F@5: {:.3f}".format(model, avg_scores[0], avg_scores[1], avg_scores[2]))

  0%|          | 0/500 [00:00<?, ?it/s]

Model: FirstPhrases P@5: 0.339 R@5: 0.207 F@5: 0.242


  0%|          | 0/500 [00:00<?, ?it/s]

Model: TextRank P@5: 0.375 R@5: 0.232 F@5: 0.269


  0%|          | 0/500 [00:00<?, ?it/s]

...

Model: MultipartiteRank P@5: 0.356 R@5: 0.212 F@5: 0.250


  0%|          | 0/500 [00:00<?, ?it/s]

Model: TfIdf P@5: 0.392 R@5: 0.244 F@5: 0.284


  0%|          | 0/500 [00:00<?, ?it/s]

Model: TopicalPageRank P@5: 0.388 R@5: 0.241 F@5: 0.279
